# Automação Web e Busca de Informações com Python

## Sobre

Nesse notebook é apresentado um projeto em python que realiza as seguintes tarefas:

0. Entender o desafio proposto
1. Buscar informações na Web
2. Atualizar a base de dados

## Desafio

> Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

> Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

## Agradecimento

Esse projeto foi desenvolvido graças ao Profº Lira e a [hashtag](https://www.hashtagtreinamentos.com/) pela realização do evento intensivão de python.


# 01: Bibliotecas Necessárias

Para desenvolver esse programa iremos precisar das seguintes bibliotecas:

1. [Pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/index.html): Para análise de dados
2. [Selenium](https://selenium-python.readthedocs.io/): Para automação WEB

Realizando a instalação no terminal:

```shell
pip install pandas
pip install selenium
```

In [1]:
import pandas as pd                             # Pandas para análise de dados
from selenium import webdriver                  # Webdriver para navegação web
from selenium.webdriver.common.keys import Keys # Teclas como ENTER

# 02: Buscando Informações na Web

Nessa etapa iremos coletar os dados da web utilizando o selenium.

Obs: O plano poderia ser mais curto, mais ele foi propositalmente alongado para demonstrar as funcionalidades do selenium.

## O plano:

1. Entrar no site do google
2. Digitar "cotação dolar"
3. Apertar ENTER para pesquisar
4. Coletar a cotação informada

Porém isso só da certo para moedas, e ao pesquisar "cotação ouro" aparece um resultado diferente que é tirado de outro site, por isso nosso plano não funciona mais 😢.

## O plano B:

1. Entrar no site do melhor cambio
2. Pesquisar e clicar no ícone "Ouro Hoje"
3. Mudar de aba do navegador
4. Coletar a cotação informada


In [2]:
nav = webdriver.Chrome()    # Inicia o driver do chrome

xpaths = {  # Armazena xpaths em um dicionário
    'google_input': '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input',
    'cotacao': '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]',
    'icone_ouro': '//*[@id="commodity-hoje"]/tbody/tr[2]/td[2]/a/img',
}

def cotacao(moeda): # Função para pegar a cotação de uma moeda
    nav.get('https://www.google.com/')  # Abre o site do google
    nav.find_element_by_xpath(xpaths['google_input']).send_keys(f'cotação {moeda}') # Escreve a pesquisa
    nav.find_element_by_xpath(xpaths['google_input']).send_keys(Keys.ENTER)         # Pesquisa a cotação
    return nav.find_element_by_xpath(xpaths['cotacao']).get_attribute('data-value') # Retorna a cotação

cotacao_dolar = cotacao('dolar')    # Pega cotação do dolar
cotacao_euro = cotacao('euro')      # Pega cotação do euro

nav.get('https://www.melhorcambio.com/')    # Abre o site do melhor cambio

aba_original = nav.window_handles[0]        # Armazena a aba para um futuro switch

nav.find_element_by_xpath(xpaths['icone_ouro']).click() # Clina no ícone do ouro

aba_nova = nav.window_handles[1]            # Armazena a aba nova

nav.switch_to.window(aba_nova)              # Muda de aba para aba nova

cotacao_ouro = nav.find_element_by_id('comercial').get_attribute('value')   # Pega o valor do ouro

cotacao_ouro = cotacao_ouro.replace(",", ".")   # Troca vírgula por ponto

print(f'Ouro:{cotacao_ouro}\nEuro:{cotacao_euro}\nDolar:{cotacao_dolar}')   # Printa as cotações

nav.quit()  # Fecha o navegador

Ouro:280.47
Euro:5.868913578000001
Dolar:4.9147


# 03: Atualizando a Base de Dados

Agora que nós conseguimos os valores atualizados das cotações, devemos fazer a atualização de nossa base de dados, para isso iremos usar o pandas. Vamos também aproveitar para fazer uma rápida checagem nos dados para verificar dados faltantes, tipagem errada ou preenchimento errado.


In [3]:
produtos = pd.read_excel('data/Produtos.xlsx')

display(produtos)               # Exibe o DataFrame
print(produtos.info())          # Exibe metadados do DataFrame
print(produtos.isna().sum())    # Exibe número de valores faltantes por coluna

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Produtos             7 non-null      object 
 1   Preço Base Original  7 non-null      float64
 2   Moeda                7 non-null      object 
 3   Cotação              7 non-null      int64  
 4   Preço Base Reais     7 non-null      float64
 5   Margem               7 non-null      float64
 6   Preço Final          7 non-null      float64
dtypes: float64(4), int64(1), object(2)
memory usage: 520.0+ bytes
None
Produtos               0
Preço Base Original    0
Moeda                  0
Cotação                0
Preço Base Reais       0
Margem                 0
Preço Final            0
dtype: int64


Após o importe e checagem da base de dados devemos atualizar os valores antigos e persistir essas mudanças em uma planilha atualizada. Obs: A tipagem retirada da web é ``string``, devemos fazer um cast para float.


In [4]:
produtos.loc[produtos['Moeda']=='Dólar', 'Cotação'] = float(cotacao_dolar)  # Atualiza cotação do dolar
produtos.loc[produtos['Moeda']=='Euro', 'Cotação'] = float(cotacao_euro)    # Atualiza cotação do euro
produtos.loc[produtos['Moeda']=='Ouro', 'Cotação'] = float(cotacao_ouro)    # Atualiza cotação do ouro

# Atualiza o preço base em real
produtos['Preço Base Reais'] = produtos['Cotação'] * produtos['Preço Base Original']
# Atualiza o preço final
produtos['Preço Final'] = produtos['Margem'] * produtos['Preço Base Reais']

display(produtos)   # Mostra tabela atualizada

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.914700,4914.650853,1.40,6880.511194
1,Carro Renault,4500.00,Euro,5.868914,26410.111101,2.00,52820.222202
2,Notebook Dell,899.99,Dólar,4.914700,4423.180853,1.70,7519.407450
3,IPhone,799.00,Dólar,4.914700,3926.845300,1.70,6675.637010
4,Carro Fiat,3000.00,Euro,5.868914,17606.740734,1.90,33452.807395
5,Celular Xiaomi,480.48,Dólar,4.914700,2361.415056,2.00,4722.830112
6,Joia 20g,20.00,Ouro,280.470000,5609.400000,1.15,6450.810000


Finalmente persistimos a mudança na tabela em disco.


In [5]:
produtos.to_excel("data/Produtos Atualizado.xlsx", index=False)